In [ ]:
!pip install -Uq "trl[peft]" trackio bitsandbytes liger-kernel wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 988.2/988.2 kB 14.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 83.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 82.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 113.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 24.4 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
hf_token = "YOUR_HUGGINGFACEHUB_TOKEN_HERE"
wanbd_token = "YOUR_WANDB_TOKEN_HERE"

In [ ]:
from huggingface_hub import login

login(token=hf_token)

In [ ]:
import wandb

wandb.login(wanbd_token)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: leviettin to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import os
os.environ["WANDB_PROJECT"] = "DPOTraining"  # name your W&B project

In [ ]:
model_id, output_dir = "meta-llama/Llama-3.2-3B-Instruct", "Llama-3.2-3B-Instruct-DPO"

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    attn_implementation="sdpa",                   # Change to Flash Attention if GPU has support
    dtype=torch.float16,                          # Change to bfloat16 if GPU has support
    device_map="auto",
    # use_cache=True,                               # Whether to cache attention outputs to speed up inference
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,                        # Load the model in 4-bit precision to save memory
        bnb_4bit_compute_dtype=torch.float16,     # Data type used for internal computations in quantization
        bnb_4bit_use_double_quant=True,           # Use double quantization to improve accuracy
        bnb_4bit_quant_type="nf4"                 # Type of quantization. "nf4" is recommended for recent LLMs
    )
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

2026-01-02 12:13:26.412023: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767356006.815183      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767356006.935172      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767356007.931812      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767356007.931852      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767356007.931855      55 computation_placer.cc:177] computation placer alr

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
adapter_model = f"JellyFush/Llama-3.2-3B-Instruct" # Replace with your HF username or organization
model = PeftModel.from_pretrained(
    base_model,
    adapter_model,
    adapter_name="policy",
    is_trainable=True
)

# ---- Load REFERENCE adapter (frozen) ----
model.load_adapter(
    adapter_model,
    adapter_name="ref",
    is_trainable=False
)

# Set the active adapter to policy for training
model.set_adapter("policy")

# Enable input gradients (required for gradient checkpointing with quantized models)
model.enable_input_require_grads()

# Enable gradient checkpointing with use_reentrant=False (required for quantized models)
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})

# Verify trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable_params:,} || All params: {all_params:,} || Trainable%: {100 * trainable_params / all_params:.2f}%")

adapter_config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/peft/config.py:165: UserWarning: Unexpected keyword arguments ['alora_invocation_tokens', 'arrow_config', 'ensure_weight_tying', 'peft_version'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/48.7M [00:00<?, ?B/s]

Trainable params: 24,313,856 || All params: 1,852,091,392 || Trainable%: 1.31%


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Download file from Google Drive
import os
import gdown

# https://drive.google.com/file/d/1ho5GuoXy0laRI450RFQ8WR9x4flz1jy9/view?usp=sharing
# Google Drive file ID from the sharing link, then change the ID
file_id = "1ho5GuoXy0laRI450RFQ8WR9x4flz1jy9"
# URL for downloading
url = f"https://drive.google.com/uc?id={file_id}"

output_path = "/content/full_dpo_dataset_2.json"

# Download the file
gdown.download(url, output_path, quiet=False)

print(f"Downloaded to {output_path}")


Downloading...
From: https://drive.google.com/uc?id=1ho5GuoXy0laRI450RFQ8WR9x4flz1jy9
To: /content/full_dpo_dataset_2.json
100%|██████████| 1.67M/1.67M [00:00<00:00, 137MB/s]

Downloaded to /content/full_dpo_dataset_2.json


In [ ]:
from datasets import load_dataset

ds = load_dataset("json", data_files="/content/full_dpo_dataset_2.json", split="train")
ds

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['question', 'accepted_response', 'rejected_response'],
    num_rows: 400
})

In [ ]:
system_prompt = """Role:
You are an expert Medical Diagnostician and Board Examiner.

Task:
Read the provided Clinical Vignette. Determine the most likely diagnosis.
You must first generate a structured clinical reasoning process inside <think> tags, followed by the final diagnosis as plain text.

Output Format:
<think>
[Explanation]
</think>

[Final Diagnosis Name]
"""

def create_dpo_messages(row):
    return {
        "prompt": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": row["question"]
            }
        ],
        "chosen": [
            {
                "role": "assistant",
                "content": row["accepted_response"]
            }
        ],
        "rejected": [
            {
                "role": "assistant",
                "content": row["rejected_response"]
            }
        ]
    }

ds_dpo = ds.map(
    create_dpo_messages,
    remove_columns=ds.column_names
)

ds_dpo


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 400
})

In [ ]:
ds_dpo[0]

{'prompt': [{'content': 'Role:\nYou are an expert Medical Diagnostician and Board Examiner.\n\nTask:\nRead the provided Clinical Vignette. Determine the most likely diagnosis.\nYou must first generate a structured clinical reasoning process inside <think> tags, followed by the final diagnosis as plain text.\n\nOutput Format:\n<think>\n[Explanation]\n</think>\n\n[Final Diagnosis Name]\n',
   'role': 'system'},
  {'content': 'A 50-year-old male physician is medically evacuated to a high-level isolation unit after developing a febrile illness while working in Sierra Leone. His initial clinical course is dominated by severe gastrointestinal symptoms requiring aggressive fluid resuscitation. However, days after admission, his condition deteriorates significantly with the onset of profound hypoxia and respiratory failure, necessitating advanced airway management and mechanical ventilation. Chest imaging reveals diffuse pulmonary infiltrates. Diagnostic bronchoscopy is performed, and molecula

In [ ]:
# This split the dataset by getting 1 row of each disease and put in the validation set
train_indices = []
val_indices = []

rows_per_disease = 5

for i in range(0, len(ds_dpo), rows_per_disease):
    chunk = list(range(i, i + rows_per_disease))

    # 1 sample → validation
    val_indices.append(chunk[0])

    # remaining samples → training
    train_indices.extend(chunk[1:])

train_dataset = ds_dpo.select(train_indices)
eval_dataset  = ds_dpo.select(val_indices)

train_dataset = train_dataset.shuffle(seed=42)

print(f"Training samples: {len(train_dataset)}")
print(f"Evaluation samples: {len(eval_dataset)}")

Training samples: 320
Evaluation samples: 80


In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
      r=16,
      lora_alpha=16,
      # lora_dropout=0.05,
      target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

In [ ]:
from trl import DPOConfig

training_args = DPOConfig(
    output_dir=output_dir,
    beta=0.1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    learning_rate=1e-6,
    num_train_epochs=1,
    save_steps=100,
    logging_steps=1,
    bf16=True,
    fp16=False,
    report_to="wandb",
    model_adapter_name="policy",
    ref_adapter_name="ref",
    max_length=2048,
    # use_liger_kernel=True,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    # Evaluation settings
    eval_strategy="steps",          # Run evaluation every eval_steps
    eval_steps=40,                  # Evaluate every 50 steps
    # eval_strategy="epoch",        # Alternative: evaluate at end of each epoch
    push_to_hub=True,
)

In [ ]:
from trl import DPOTrainer

trainer = DPOTrainer(
    model=model,
    ref_model=None,           # Using adapter-based reference via ref_adapter_name
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    # peft_config=peft_config,  # Don't pass this when using existing adapters
    processing_class=tokenizer,
)

Extracting prompt in train dataset:   0%|          | 0/320 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/320 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/320 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/80 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/80 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/80 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
40,0.350000,0.306412,0.322087,-0.721672,1.000000,1.043760,-439.709290,-393.535065,-0.580774,-0.115414
80,0.105000,0.155963,0.508115,-1.331572,1.000000,1.839687,-437.849030,-399.634064,-0.596159,-0.127493
120,0.116700,0.102654,0.609010,-1.732829,1.000000,2.341839,-436.840088,-403.646637,-0.606984,-0.136768
160,0.076500,0.087870,0.655627,-1.863619,1.000000,2.519246,-436.373871,-404.954498,-0.609569,-0.138619


No files have been modified since last commit. Skipping to prevent empty commit.


TrainOutput(global_step=160, training_loss=0.23445073124021293, metrics={'train_runtime': 11256.027, 'train_samples_per_second': 0.028, 'train_steps_per_second': 0.014, 'total_flos': 0.0, 'train_loss': 0.23445073124021293, 'epoch': 1.0})

In [ ]:
# Save the trained adapter locally
trainer.save_model(output_dir)

# Save the PEFT adapter separately
model.save_pretrained(f"{output_dir}")

# Push to Hugging Face Hub using trainer (recommended - handles model + tokenizer)
trainer.push_to_hub(f"{output_dir}", token=hf_token)

print(f"Model saved to {output_dir} and {output_dir}")
print(f"Pushed to Hugging Face Hub: JellyFush/{output_dir}")

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


Model saved to Llama-3.2-3B-Instruct-DPO and Llama-3.2-3B-Instruct-DPO
Pushed to Hugging Face Hub: JellyFush/Llama-3.2-3B-Instruct-DPO
